# 🦤 Dodo Source

[doit](https://github.com/pydoit/doit) is a simple, yet powerful task execution tool,
written in Python. The `DoitSource` demonstrates.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements-dodo.txt

In [ ]:
import asyncio
import functools
import subprocess
import threading
from collections import deque

import ipylab as L
import ipywidgets as W
import traitlets as T
from ipydatagrid import DataGrid
from ipyforcegraph import behaviors as B
from ipyforcegraph import graphs as G
from ipyforcegraph.sources.dodo import DodoSource

DODO = "🦤"

## Create a Panel

Creating a single panel in the JupyterLab `main` area next to the notebook we're working
on makes it easy to see how the application develops.

In [ ]:
lab = L.JupyterFrontEnd()
split = L.SplitPanel(
    [], orientation="vertical", layout=dict(height="100%", overflow="hidden")
)
app_style = W.HTML(
    """<style>
.ipfg-dodo {
    --jp-widgets-inline-width-short: auto;
}
</style>"""
)

app = W.VBox(
    [app_style, split],
    _dom_classes=["ipfg-dodo"],
    layout=dict(display="flex", flex="1", flex_flow="column"),
)

panel = L.Panel([app], layout=dict(overflow="hidden", height="100%"))
panel.title.label = DODO

In [ ]:
def show_app(area, **options):
    lab.shell.add(panel, area, options)
    return panel

In [ ]:
if __name__ == "__main__":
    show_app("main", mode="split-right")

## Create the Graph

We know we want to show a graph.

In [ ]:
fg = G.ForceGraph(layout=dict(width="100%", height="100%", flex="3"))
split.children = [fg]

## Create the `DodoSource`

A `DodoSource` needs to know its `project_root` in order to find your `dodo.py`, and
establish the right current working directory.

> While `doit` has pluggable _loaders_, `DodoSource` only supports discovering a
> `dodo.py`

In [ ]:
source = DodoSource(project_root="..")
T.dlink((source, "project_root"), (panel.title, "label"), lambda x: f"{DODO}: {x.name}")
fg.source = source

> The app should now show a big pile of `nodes`, based on the underlying
> `pandas.DataFrame`, to be explored more fully below.

## Add `Tooltip` Behaviors

Tooltips provide a quick way to inspect parts of the graph.

In [ ]:
node_tooltip = B.NodeTooltip(
    B.Nunjucks("<b>[{{ node.type }}]</b> {{ node.name }}<br/>{{ node.doc }}")
)
link_tooltip = B.LinkTooltip(
    B.Nunjucks(
        "{{ link.source.name }}<br/><b>[{{ link.type }}]</b> <br/>{{ link.target.name }}"
    )
)
fg.behaviors = [node_tooltip, link_tooltip]

## Toggle Graph Features

`doit` task graphs can quickly get _large_. Some graph features decrease (or increase)
the number of nodes or edges.

In [ ]:
button_bar_style = dict(layout=dict(height="2.5em", overflow="hidden"))
view_toggles = W.HBox(**button_bar_style)
view_toggles.layout.flex_flow = "row-reverse"
app.children = [app_style, view_toggles, split]

#### Directories

Directories can't be acted upon by `doit`, generally, and add a lot of extra edges, but
can be useful for more directly visualizing the project layout.

In [ ]:
show_directories = W.ToggleButton(icon="folder", tooltip="Show Directories")
T.dlink((show_directories, "value"), (source, "show_directories"))
view_toggles.children = [*view_toggles.children, show_directories]

> Now, hovering over each of the nodes and edges should show some data about them.

## Create some `DataGrid`s

[ipydatagrid](https://github.com/bloomberg/ipydatagrid) provides a rich `DataGrid`
class, which also speaks `pandas.DataFrame`. Since even a relatively small `doit` task
graph can be quite large, displaying the raw data in a grid provides a more readly
inspectable (and filterable) view.

In [ ]:
grid_opts = dict(
    layout=dict(height="100%", max_height="100%", min_height="100%"),
)
node_grid = DataGrid(fg.source.nodes, **grid_opts)
link_grid = DataGrid(fg.source.links, **grid_opts)
grid_panel = L.SplitPanel(
    [node_grid, link_grid],
    orientation="horizontal",
    layout=dict(overflow="hidden", height="400px", min_height="400px"),
)
split.children = [fg, grid_panel]

> The app should now contain the graph, with two grids below it.

## Add Node and Link Selection

Both `ForceGraph` and `DataGrid` support the concept of _selection_.

In [ ]:
node_grid.selection_mode = "row"
link_grid.selection_mode = "row"
node_selection = B.NodeSelection()
link_selection = B.LinkSelection()
fg.behaviors = [*fg.behaviors, node_selection, link_selection]

> Nodes, links, and grid rows can now all be selected

## Link the Selections

While the graph and grids are now selectable, they are not expressed in the same format,
and there is no relationship between them. With `traitlets.dlink`, we can create a
semi-stable, bidrectional behavior between them.

In [ ]:
def link_grid_and_graph(grid, graph_selection, source, kind):
    last_selected = set()

    def on_grid_select(*x):
        nonlocal last_selected
        if not grid.selections:
            graph_selection.selected = []
            return
        visible = grid.get_visible_data()
        selected = []
        for selection in grid.selections:
            for i in range(selection["r1"], selection["r2"] + 1):
                data_idx = int(visible.iloc[i].name)
                selected += [data_idx]
        if last_selected != set(selected):
            last_selected = set(selected)
            graph_selection.selected = sorted(set(selected))

    grid.observe(on_grid_select, ["selections"])

    def on_graph_select(*x):
        nonlocal last_selected
        last_selected = set(graph_selection.selected)
        if not graph_selection.selected:
            grid.selections = []
            return
        visible = grid.get_visible_data()
        visible_index = visible.index.to_list()
        c2 = visible.shape[1] - 1
        selections = []
        getattr(source, kind)
        for index in graph_selection.selected:
            try:
                r1 = int(visible_index.index(index))
            except Exception as err:  # noqa
                continue
            selection = {"r1": r1, "r2": r1, "c1": 0, "c2": c2}
            selections += [selection]
        grid.selections = selections

    graph_selection.observe(on_graph_select, ["selected"])

In [ ]:
link_grid_and_graph(node_grid, node_selection, source, "nodes")
link_grid_and_graph(link_grid, link_selection, source, "links")

## Customize some Shapes

In [ ]:
node_shape = B.NodeShapes()
fg.behaviors = [*fg.behaviors, node_shape]

### Use node size for type

In [ ]:
node_shape.size = B.Nunjucks(
    "{% if node.type == 'task' %}"
    "10"
    "{% elif node.type == 'file' %}"
    "{% if node.exists %}1{% else %}2{% endif %}"
    "{% endif %}"
)

## Use color for status

In [ ]:
node_shape.color = B.Nunjucks(
    "{% if node.type == 'task' %}"
    "{% if node.status == ['error'] %}rgba(255,0,0,0.5){% endif %}"
    "{% elif node.type == 'file' %}"
    "{% if node.exists %}rgba(0,0,0,0.75){% else %}rgba(150,150,0,0.75){% endif %}"
    "{% elif node.type == 'directory' %}"
    "{% if node.exists %}rgba(0,0,0,0.25){% else %}rgba(150,150,0,0.25){% endif %}"
    "{% endif %}"
)

## Add a button bar

In [ ]:
action_buttons = W.HBox(**button_bar_style)
app.children = [app_style, view_toggles, split, action_buttons]

### A button helper

This helper will react to the state of a given graph selection, showing when a command
is relevant.

In [ ]:
def add_a_button(label, icon, row_filter, selection, items):
    button = W.Button(description=label, icon=icon)
    action_buttons.children = [*action_buttons.children, button]

    def on_selection(*x):
        selected = selection.selected
        filtered = []
        df = getattr(source, items)
        for i in selected:
            item = df.loc[i]
            keep = True
            for key, value in row_filter.items():
                if getattr(item, key) not in value:
                    keep = False
                    break
            if keep:
                filtered += [i]
        with button.hold_sync():
            if filtered:
                button.disabled = False
                button.button_style = "primary"
                button.description = f"{label} ({len(filtered)})"
            else:
                button.disabled = True
                button.button_style = ""
                button.description = label

    selection.observe(on_selection)
    on_selection()
    return button


add_node_button = functools.partial(
    add_a_button, selection=node_selection, items="nodes"
)
add_link_button = functools.partial(
    add_a_button, selection=link_selection, items="links"
)

### Refresh the Graph

You might work somewhere else on the tasks, and want to reload the data.

In [ ]:
def refresh():
    source.refresh()
    node_grid.data = source.nodes
    link_grid.data = source.links

In [ ]:
refresh_button = W.Button(description="Refresh", icon="refresh")
action_buttons.children = [refresh_button, *action_buttons.children]

In [ ]:
refresh_button.on_click(lambda x: refresh())

### View Files

`ipylab` exposes the
[JupyterLab command](https://jupyterlab.readthedocs.io/en/stable/user/commands.html)
system, which allows for changing the state of the running client application from the
kernel. T

In [ ]:
view_file_button = add_node_button(
    "Open Files", "folder-open", {"type": ["file"], "exists": [True]}
)


def on_view_click():
    cmd_id = "filebrowser:open-path"
    for i in node_selection.selected:
        node = source.nodes.iloc[i]
        if node.type == "file" and node.exists:
            node.id.split("file:")[1]
            lab.commands.execute(cmd_id, {"path": node["name"]})


view_file_button.on_click(lambda *x: on_view_click())

### Run Tasks

In [ ]:
run_task_button = add_node_button("Run", "play", {"type": ["task"]})

In [ ]:
def run_tasks(tasks, lines):
    args = ["doit", *tasks]
    lines.append(("stdout", " ".join([">>>", *args, "\n"])))
    proc = subprocess.Popen(
        args,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        bufsize=1,
        universal_newlines=True,
        cwd=str(source.project_root),
    )
    streams = {"stdout": proc.stdout, "stderr": proc.stderr}

    def watch_stream(stream_name, stream, lines):
        line = stream.readline()
        while line:
            lines.append((stream_name, line))
            line = stream.readline()

    for stream_name, stream in streams.items():
        thread = threading.Thread(None, watch_stream, args=(stream_name, stream, lines))
        thread.start()

    proc.wait()

    lines.append(("stdout", f"return code: {proc.returncode}"))
    return proc.returncode

In [ ]:
async def run_tasks_async(tasks, output_panel):
    lines = deque()
    task = asyncio.get_running_loop().run_in_executor(None, run_tasks, tasks, lines)
    while not task.done():
        output = output_panel.children[0]
        await asyncio.sleep(0.01)
        while lines:
            stream, line = lines.pop()
            if stream == "stderr":
                output.append_stderr(line)
            else:
                output.append_stdout(line)
            if len(output.outputs) >= 100:
                output = W.Output()
                output_panel.children = [output, *output_panel.children]
    refresh()
    run_task_button.button_style = "primary"

In [ ]:
def on_run_click():
    tasks = []
    for i in node_selection.selected:
        node = source.nodes.iloc[i]
        if node.type == "task":
            tasks += [node["name"]]
    if tasks:
        output = W.Output()
        style = W.HTML(
            """<style>
            .ipfg-stream { display: flex; flex-direction: column-reverse; }
            .ipfg-stream .jp-OutputPrompt {display: none;}
        </style>"""
        )
        output_panel = L.Panel(
            [output, style],
            _dom_classes=["ipfg-stream"],
            layout=dict(overflow="scroll"),
        )
        output_panel.title.label = DODO + (" ".join(tasks))
        lab.shell.add(output_panel, "main", {"mode": "split-right"})

        run_task_button.button_style = "warning"
        asyncio.create_task(run_tasks_async(tasks, output_panel))

In [ ]:
run_task_button.on_click(lambda *x: on_run_click())

### Forget Tasks

If a task _thinks_ it's up-to-date, but you _know_ it isn't, it can be useful to
_forget_ a task, forcing it (and any dependent tasks) to be re-run.

In [ ]:
forget_task_button = add_node_button("Forget", "eraser", {"type": ["task"]})

In [ ]:
def forget_tasks():
    tasks = []
    for i in node_selection.selected:
        node = source.nodes.iloc[i]
        if node.type == "task":
            tasks += [node["name"]]
    if not tasks:
        return
    forget_task_button.button_style = "warning"
    try:
        subprocess.check_call(["doit", "forget", *tasks], cwd=str(source.project_root))
        forget_task_button.button_style = "primary"
    except Exception:
        forget_task_button.button_style = "danger"
    refresh()

In [ ]:
forget_task_button.on_click(lambda *x: forget_tasks())